# Reconstrucción de la Curva de Tipos de Interés

Este notebook implementa la reconstrucción de la curva de tipos de interés (ETTI) usando bonos del Tesoro de Estados Unidos mediante:

1. **Bootstrapping** para obtener tasas spot a partir de precios de mercado
2. **Modelo Nelson-Siegel** para ajustar una curva suave a los datos observados

## Metodología

El proceso consta de dos etapas principales:
- **Bootstrapping**: Construcción de la curva spot a partir de bonos zero-coupon y bonos con cupón
- **Calibración Nelson-Siegel**: Ajuste de una curva paramétrica usando mínimos cuadrados ordinarios (OLS)


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt


## 1. Carga de Datos

Cargamos el archivo Excel con las cotizaciones de bonos del Tesoro de Estados Unidos.


In [ ]:
file_path = "GovernmentBondPrices_UnitedStates.xlsx"
df = pd.read_excel(file_path)


## 2. Normalización de Columnas

Renombramos las columnas para facilitar el procesamiento.


In [ ]:
df = df.rename(columns={
    "Cpn": "Coupon_rate",
    "Mat Date": "Maturity_Date",
    "Date": "Valuation_Date"
})


## 3. Conversión de Precios US (Formato 32nds)

Los bonos con cupón del Tesoro de Estados Unidos cotizan en formato de puntos y treintaidosavos. Por ejemplo:
- `99*19` = 99 + 19/32 = 99.59375
- `98*28¾` = 98 + 28.75/32 = 98.8984

**Fórmula de conversión:**
$$P_{decimal} = P_{entero} + \frac{P_{32avos} + P_{fracción}}{32}$$

donde $P_{fracción}$ puede ser 0, ¼, ½, ¾, etc.


In [ ]:
UNICODE_FRACTIONS = {
    '¼': 0.25, '½': 0.5, '¾': 0.75,
    '⅛': 0.125, '⅜': 0.375,
    '⅝': 0.625, '⅞': 0.875
}

def us_price_to_decimal(price):
    if isinstance(price, (int, float)):
        return float(price)
    try:
        main, frac = price.split('*')
        integer = float(main)
        match = re.match(r"(\d+)(.*)", frac)
        thirty = int(match.group(1))
        remainder = match.group(2)
        extra = UNICODE_FRACTIONS.get(remainder, 0.0)
        return integer + (thirty + extra) / 32
    except:
        return np.nan


## 4. Normalización de Cupones

Normalizamos las tasas de cupón para que estén en porcentaje (0-100). Si el valor es ≤ 1, asumimos que está en decimal y lo multiplicamos por 100.


In [ ]:
def normalize_coupon(c):
    if pd.isna(c):
        return 0.0
    if isinstance(c, str):
        c = c.replace('%', '').strip()
    c = float(c)
    return c * 100 if c <= 1 else c

df["Coupon_rate"] = df["Coupon_rate"].apply(normalize_coupon)


## 5. Detección del Tipo de Cotización

Los datos pueden estar cotizados en **precio** o en **yield**. Detectamos el tipo:
- Si contiene `*`, es precio en formato 32nds
- Si el valor numérico < 30, probablemente es yield (%)
- Si el valor ≥ 30, probablemente es precio


## 6. Tiempo hasta Vencimiento

Calculamos el tiempo hasta vencimiento en años usando días exactos:

$$T = \frac{\text{Días hasta vencimiento}}{365.25}$$

Usamos 365.25 para tener en cuenta los años bisiestos.


In [ ]:
def detect_quote_type(x):
    if isinstance(x, str) and '*' in x:
        return "Price"
    try:
        val = float(x)
        return "Yield" if val < 30 else "Price"
    except:
        return np.nan

df["Bid_type"] = df["Bid"].apply(detect_quote_type)
df["Ask_type"] = df["Ask"].apply(detect_quote_type)

df["Quote_type"] = np.where(
    (df["Bid_type"] == "Yield") | (df["Ask_type"] == "Yield"),
    "Yield",
    "Price"
)

df.loc[df["Quote_type"] == "Price", "Quoted_value"] = (
    df.loc[df["Quote_type"] == "Price", ["Bid", "Ask"]]
      .map(us_price_to_decimal)
      .mean(axis=1)
)

df.loc[df["Quote_type"] == "Yield", "Quoted_value"] = (
    df.loc[df["Quote_type"] == "Yield", ["Bid", "Ask"]]
      .astype(float)
      .mean(axis=1)
)


In [ ]:
df["Maturity_Date"] = pd.to_datetime(df["Maturity_Date"])
df["Valuation_Date"] = pd.to_datetime(df["Valuation_Date"])
df["T"] = (df["Maturity_Date"] - df["Valuation_Date"]).dt.days / 365.25
df = df[df["T"] > 0]


## 8. Bootstrapping de la Curva Spot

El bootstrapping construye la curva de tasas spot a partir de los precios de mercado. El proceso depende del tipo de instrumento:

### 8.1 Bonos Zero-Coupon

**Si cotiza en yield:**
$$r(T) = \frac{YTM}{100}$$

**Si cotiza en precio:**
$$r(T) = -\frac{\ln(P/100)}{T}$$

donde $P$ es el precio y $T$ es el tiempo hasta vencimiento.

### 8.2 Bonos con Cupón

**Si cotiza en yield:**
Usamos el YTM directamente como proxy del spot rate (según el enunciado):
$$r(T) = \frac{YTM}{100}$$

**Si cotiza en precio:**
Aplicamos bootstrapping iterativo:

1. Calculamos el valor presente de los cupones conocidos usando spots interpolados:
   $$\text{PV}_{cupones} = \sum_{k=1}^{n-1} c \cdot e^{-r(t_k) \cdot t_k}$$
   
   donde $c$ es el cupón semestral y $r(t_k)$ es el spot rate interpolado en $t_k = k/freq$.

2. Calculamos el spot rate para el último flujo:
   $$P - \text{PV}_{cupones} = (F + c) \cdot e^{-r(T) \cdot T}$$
   
   Despejando:
   $$r(T) = -\frac{\ln\left(\frac{P - \text{PV}_{cupones}}{F + c}\right)}{T}$$
   
   donde $F$ es el valor facial (100) y $c$ es el último cupón.


## 7. Clasificación de Instrumentos

Clasificamos los instrumentos en:
- **Zero**: Bonos zero-coupon (cupón = 0%)
- **Coupon**: Bonos con cupón (cupón > 0%)


In [ ]:
df["Instrument_class"] = np.where(df["Coupon_rate"] == 0, "Zero", "Coupon")

clean_df = df[[
    "RIC", "Coupon_rate", "Quoted_value",
    "Quote_type", "T", "Instrument_class"
]].dropna().sort_values("T").reset_index(drop=True)

print("\n" + "="*60)
print("DIAGNÓSTICO DE DATOS")
print("="*60)
print(f"\nTotal de instrumentos después de limpieza: {len(clean_df)}")
print(f"\nPor tipo de instrumento:")
print(clean_df["Instrument_class"].value_counts())
print(f"\nPor tipo de cotización:")
print(clean_df["Quote_type"].value_counts())
print(f"\nBonos con cupón por tipo de cotización:")
if len(clean_df[clean_df["Instrument_class"] == "Coupon"]) > 0:
    print(clean_df[clean_df["Instrument_class"] == "Coupon"]["Quote_type"].value_counts())
print(f"\nRango de vencimientos: {clean_df['T'].min():.2f} - {clean_df['T'].max():.2f} años")
print("="*60 + "\n")


### Resultados del Bootstrapping


In [ ]:
spot_curve_list = []
spot_curve_dict = {}

def interp_spot(t):
    if not spot_curve_dict:
        return 0.0
    Ts = np.array(sorted(spot_curve_dict.keys()))
    rs = np.array([spot_curve_dict[x] for x in Ts])
    if t <= Ts[0]:
        return rs[0]
    if t >= Ts[-1]:
        return rs[-1]
    return np.interp(t, Ts, rs)

for _, row in clean_df.iterrows():
    T = row["T"]
    P = row["Quoted_value"]
    cpn = row["Coupon_rate"]

    if row["Instrument_class"] == "Zero":
        if row["Quote_type"] == "Yield":
            r = P / 100
        else:
            r = -np.log(P / 100) / T
        spot_curve_list.append((T, r, "Zero"))
        if T in spot_curve_dict:
            spot_curve_dict[T] = (spot_curve_dict[T] + r) / 2
        else:
            spot_curve_dict[T] = r
        continue

    face = 100
    freq = 2
    c = face * cpn / 100 / freq
    n = int(np.round(T * freq))
    
    if row["Quote_type"] == "Yield":
        r_T = P / 100
        spot_curve_list.append((T, r_T, "Coupon"))
        if T in spot_curve_dict:
            spot_curve_dict[T] = (spot_curve_dict[T] + r_T) / 2
        else:
            spot_curve_dict[T] = r_T
        continue
    
    pv_known = 0.0
    for k in range(1, n):
        t = k / freq
        if t < T:
            r_t = interp_spot(t)
            pv_known += c * np.exp(-r_t * t)
    
    pv_final = P - pv_known
    
    if pv_final > 0 and pv_final < (face + c) * 2:
        r_T = -np.log(pv_final / (face + c)) / T
        spot_curve_list.append((T, r_T, "Coupon"))
        if T in spot_curve_dict:
            spot_curve_dict[T] = (spot_curve_dict[T] + r_T) / 2
        else:
            spot_curve_dict[T] = r_T


## 10. Visualización

Visualizamos los spots observados (diferenciando bonos zero-coupon y con cupón) junto con la curva Nelson-Siegel ajustada.


In [ ]:
boot_df = pd.DataFrame(spot_curve_list, columns=["T", "Spot", "Instrument_type"]).sort_values("T")

print("\n" + "="*60)
print("RESULTADOS DEL BOOTSTRAPPING")
print("="*60)
print(f"\nTotal de spots generados: {len(boot_df)}")
print(f"\nSpots únicos por T: {boot_df['T'].nunique()}")
print(f"\nRango de vencimientos con spots: {boot_df['T'].min():.2f} - {boot_df['T'].max():.2f} años")
print(f"\nRango de spot rates: {boot_df['Spot'].min()*100:.4f}% - {boot_df['Spot'].max()*100:.4f}%")
print("="*60 + "\n")


## 9. Calibración del Modelo Nelson-Siegel

El modelo Nelson-Siegel parametriza la curva de tipos de interés mediante:

$$y(\tau) = \beta_0 + \beta_1 \cdot f_1(\tau) + \beta_2 \cdot f_2(\tau)$$

donde las funciones de carga son:

$$f_1(\tau) = \frac{1 - e^{-\tau/\lambda}}{\tau/\lambda}$$

$$f_2(\tau) = f_1(\tau) - e^{-\tau/\lambda} = \frac{1 - e^{-\tau/\lambda}}{\tau/\lambda} - e^{-\tau/\lambda}$$

### Interpretación de los parámetros:

- **$\beta_0$ (Level)**: Nivel asintótico de la curva (tasa de largo plazo)
- **$\beta_1$ (Slope)**: Pendiente de la curva (diferencia entre corto y largo plazo)
- **$\beta_2$ (Curvature)**: Curvatura o "joroba" en el medio plazo
- **$\lambda$ (Decay)**: Parámetro de decaimiento que controla la velocidad de transición

### Método de Estimación: Grid Search + OLS

Como el modelo es **lineal en $\beta$ pero no lineal en $\lambda$**, usamos:

1. **Grid search** sobre $\lambda$ en el rango [0.1, 5.0]
2. Para cada $\lambda$, estimamos $\beta$ mediante **mínimos cuadrados ordinarios (OLS)**

El modelo se puede escribir en forma matricial:

$$\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \mathbf{u}$$

donde:
- $\mathbf{y}$ es el vector de spots observados
- $\mathbf{X} = [\mathbf{1}, \mathbf{f}_1, \mathbf{f}_2]$ es la matriz de diseño
- $\boldsymbol{\beta} = [\beta_0, \beta_1, \beta_2]^T$ son los parámetros a estimar

La solución OLS es:

$$\hat{\boldsymbol{\beta}} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

Seleccionamos el $\lambda$ que minimiza la suma de errores al cuadrado (SSE):

$$\text{SSE} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$


In [ ]:
def ns_functions(tau, lam):
    ratio = tau / lam
    f1 = np.where(ratio < 1e-10, 1.0, (1 - np.exp(-ratio)) / ratio)
    f2 = f1 - np.exp(-ratio)
    return f1, f2

def nelson_siegel_curve(tau, beta0, beta1, beta2, lam):
    f1, f2 = ns_functions(tau, lam)
    return beta0 + beta1 * f1 + beta2 * f2

tau_obs = boot_df["T"].values
y_obs = boot_df["Spot"].values

lambda_grid = np.linspace(0.1, 5.0, 100)
best_sse = np.inf
best_params = None
best_lam = None

for lam in lambda_grid:
    f1, f2 = ns_functions(tau_obs, lam)
    X = np.column_stack([np.ones_like(tau_obs), f1, f2])
    betas, residuals, rank, s = np.linalg.lstsq(X, y_obs, rcond=None)
    y_fit = X @ betas
    sse = np.sum((y_obs - y_fit)**2)
    
    if sse < best_sse:
        best_sse = sse
        best_params = betas
        best_lam = lam

beta0_ols, beta1_ols, beta2_ols = best_params
y_fit_ols = nelson_siegel_curve(tau_obs, beta0_ols, beta1_ols, beta2_ols, best_lam)
sse_ols = np.sum((y_obs - y_fit_ols)**2)


In [ ]:
T_grid = np.linspace(0.05, boot_df["T"].max(), 300)
curve = nelson_siegel_curve(T_grid, beta0_ols, beta1_ols, beta2_ols, best_lam)

plt.figure(figsize=(8,5))

zero_bonds = boot_df[boot_df["Instrument_type"] == "Zero"]
coupon_bonds = boot_df[boot_df["Instrument_type"] == "Coupon"]

if len(zero_bonds) > 0:
    plt.scatter(zero_bonds["T"], 100 * zero_bonds["Spot"], 
                color='blue', label="Zero-coupon bonds", alpha=0.7, s=50)

if len(coupon_bonds) > 0:
    plt.scatter(coupon_bonds["T"], 100 * coupon_bonds["Spot"], 
                color='lightgreen', label="Coupon bonds", alpha=0.7, s=50)

plt.plot(T_grid, 100 * curve, label="Nelson–Siegel (fitted)", linewidth=2, color='red')

plt.xlabel("Maturity (years)")
plt.ylabel("Spot rate (%)")
plt.title("Bootstrapped Yield Curve + Nelson–Siegel OLS")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## 11. Resultados de la Calibración

Mostramos los parámetros estimados del modelo Nelson-Siegel y el error de ajuste (SSE).


In [ ]:
print("\n" + "="*50)
print("        Nelson–Siegel Calibration Results")
print("="*50)
print("\n[ OLS with Lambda Grid Search ]")
print(f"  • beta₀ (level)     : {beta0_ols:>12.6f}")
print(f"  • beta₁ (slope)     : {beta1_ols:>12.6f}")
print(f"  • beta₂ (curvature) : {beta2_ols:>12.6f}")
print(f"  • λ (decay factor)  : {best_lam:>12.6f}")
print(f"  • SSE               : {sse_ols:>12.6e}")
print("="*50 + "\n")
